In [4]:
import os
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader

api_key = os.environ["OPENAI_API_KEY"]



### Step 1: Loading the documents 

In [7]:
loader = PyPDFLoader("docs/FAA_Engine-Maintainance.pdf")

doc = loader.load()

In [13]:
print(doc[10])

page_content='10-11
Figure 10-11. Checking a piston pin for bends. 
checked for side clearance with a feeler gauge to determine 
the amount of wear in the grooves. Examine the piston pin 
for scoring, cracks, excessive wear, and pitting. Check the 
clearance between the piston pin and the bore of the piston 
pin bosses using a telescopic gauge and a micrometer. Use 
the magnetic particle method to inspect the pin for cracks. 
Since the pins are often case hardened, cracks show up inside 
the pin more often than they do on the outside. Check the pin 
for bends using V-blocks and a dial indicator on a surface 
plate. [Figure 10-11] Measure the fit of the plugs in the 
pin. In many cases, the pistons and piston pins are routinely 
replaced at overhaul.
Valves & Valve Springs
The locations for checking runout and edge thickness of the 
valves are shown in Figure 10-12. Measure the edge thickness 
of valve heads. If, after re-facing, the edge thickness is less 
than the limit specified by t